In [ ]:
import os
os.chdir("/home/149/ab8992/tasman-tides")
# from recipes import surfacespeed
from importlib import reload
import ttidelib as tt
from ttidelib import logmsg
import shutil
import subprocess
import dask
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
experiment,outputs = "full-20","output*"
from dask.distributed import Client, default_client
from pathlib import Path
client = Client(threads_per_worker = 1,n_workers = 50)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 50
Total threads: 50,Total memory: 416.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:33739,Workers: 50
Dashboard: /proxy/8787/status,Total threads: 50
Started: Just now,Total memory: 416.00 GiB
Comm: tcp://127.0.0.1:33361,Total threads: 1
Dashboard: /proxy/37655/status,Memory: 8.32 GiB
Nanny: tcp://127.0.0.1:44729,


2025-05-23 10:31:18,893 - distributed.nanny - WARNING - Restarting worker
2025-05-23 10:31:18,902 - distributed.nanny - WARNING - Restarting worker
2025-05-23 10:31:18,966 - distributed.nanny - WARNING - Restarting worker
2025-05-23 10:31:18,970 - distributed.nanny - WARNING - Restarting worker
2025-05-23 10:31:19,271 - distributed.nanny - WARNING - Restarting worker
2025-05-23 10:31:19,276 - distributed.nanny - WARNING - Restarting worker
2025-05-23 10:31:20,289 - distributed.nanny - WARNING - Restarting worker
2025-05-23 10:31:20,295 - distributed.nanny - WARNING - Restarting worker


## Test improving collect data so it only looks at the right output files for time slice

In [ ]:
#!/usr/bin/env python3
import os
from datetime import timedelta
from dask.distributed import Client,default_client
from dask import delayed, compute
import filtering
from pathlib import Path
home = Path("/home/149/ab8992/tasman-tides")
gdata = Path("/g/data/nm03/ab8992")
import numpy as np
import xarray as xr
os.chdir(home)
import warnings
warnings.filterwarnings("ignore")
import ttidelib as tt
import argparse
def startdask(nthreads = 1):
    try:
    # Try to get the existing Dask client
        client = default_client()
        print(client)
    except ValueError:
        # If there's no existing client, create a new one
        client = Client(
            threads_per_worker = nthreads,n_workers = 20)
        print(client)
    return client

def save_data_for_filter(expt,zl,t0,tmpstorage,sample_window = 250): ## sample window is the same for each offset. Why? inefficient loading extra times

    client = startdask()
    retry = 0
    while retry < 20:
        print(t0 - sample_window // 2,t0 + sample_window // 2 + sample_window % 2)
        try:
            rawdata = tt.collect_data(
                expt,
                rawdata = ["u","v","rho"],
                timerange=(
                    t0 - sample_window // 2,t0 + sample_window // 2 + sample_window % 2
                    )).isel(zl = zl)#.load() had been loading here for some reason?
            retry = 100
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Retry loading data")
            retry +=1
    attrs = {
        "u": rawdata.u.attrs,
        "v": rawdata.v.attrs,
        "time": rawdata.time.attrs,
        "xb": rawdata.xb.attrs,
        "yb": rawdata.yb.attrs,
        "zl": rawdata.zl.attrs,
        "base" : rawdata.attrs
    }
    ## Remove zl to make data properly 2D
    rawdata.u.attrs = {}
    rawdata.v.attrs = {}
    rawdata.zl.attrs = {}
    rawdata.time.attrs = {}
    rawdata.yb.attrs = {}
    rawdata.xb.attrs = {}
    rawdata = rawdata.assign_coords({
        "time":rawdata.time * 3600,
        "xb":rawdata.xb * 1000,
        "yb":rawdata.yb * 1000})
    rawdata = rawdata.drop_vars(["bathy","lat","lon"]) 
    if not os.path.exists(tmpstorage):
        os.makedirs(tmpstorage)
    print("Saving data to temporary storage")


    # for i in range(0,rawdata.zl.size):
    out = xr.merge(
        [
            rawdata.u,
            rawdata.v,
            rawdata.rho - rawdata.rho.mean("time"), # density anomaly
        ]
    )
    print("Raw saved data times")
    print(out.time.values[0],out.time.values[-1])
    for i in range(0,96):
        out.isel(zl = [i]).to_netcdf(tmpstorage + f"/raw_{i}.nc",mode="w")
    print("done")
    client.close() ## Have to close dask client or it messes up the filtering package
    return attrs


def lagrange_filter(zl,tmpstorage,filter_range,sample_window = 250):

    """
    Apply the Lagrange filter to the input data. This is a wrapper around the LagrangeFilter class in filtering.py
    Saves the outputs to `postprocessing/expt/lfiltered/t0-<t0>/filtered_<zl>.nc` with a separate file for each z level.
    Inputs:
        expt: str, the experiment to process
        zl: int, the z level(s) to process
        t0: int, the middle time of the time slice we care about
    """
    os.chdir(tmpstorage)
    FastCutoff = 2*np.pi/(9.8*3600)  ## Same as tolerences for bandpass in Waterhouse 2018
    SlowCutoff = 2*np.pi/(15*3600)


    data_map = {
            "U":tmpstorage + f"/raw_{zl}.nc",
            "V":tmpstorage + f"/raw_{zl}.nc",
            "rho":tmpstorage + f"/raw_{zl}.nc",
        }
    if os.path.exists(tmpstorage + f"/Filtered_{zl}.nc"):
        os.remove(tmpstorage + f"/Filtered_{zl}.nc")


    BandpassFilter = filtering.LagrangeFilter(
        tmpstorage + f"/Filtered_{zl}", ## Save intermediate output to temporary storage
        data_map, 
        {"U":"u","V":"v","rho":"rho"}, 
        {"lon":"xb","lat":"yb","time":"time"},
        sample_variables=["U","V","rho"], mesh="flat",frequency = np.array([SlowCutoff,FastCutoff]),filter_type = "bandpass",
        advection_dt =timedelta(minutes=5).total_seconds(),
        window_size = timedelta(hours=48).total_seconds(),
    )

    ## Ensure we take times either side of the point of interest
    BandpassFilter(times = filter_range) 

    ## Re add zl as dimension
    zl_value = float(xr.open_dataset("/g/data/nm03/ab8992/outputs/zl.nc").isel(zl = zl).zl.values)
    FilteredData = xr.open_dataset(tmpstorage + f"/Filtered_{zl}.nc").expand_dims({"zl":[zl_value]})
    FilteredData = FilteredData.assign_coords({
        "time":FilteredData.time / 3600,
        "xb":FilteredData.xb / 1000,
        "yb":FilteredData.yb / 1000})
    for i in FilteredData.data_vars:
        FilteredData = FilteredData.rename({i:i.split("_")[1].lower()})

    # outfolder = Path(f"/g/data/nm03/ab8992/postprocessed/{expt}/lfiltered/t0-{t0}")
    outfolder = Path(tmpstorage) / "output"

    if not os.path.exists(outfolder):
        os.makedirs(outfolder)
    FilteredData.to_netcdf(outfolder / f"Filtered_{zl}.nc",mode="w")
    return 


def lfilter_tidyup(tmpstorage,outputfolder,offset = -1):
    """Necessary tidying up of metadata and units for filtered outputs"""

    if offset == -1:
        offset = ""
    data = xr.open_mfdataset(tmpstorage + f"/output/Filtered_*.nc",combine="by_coords")
    data.zl.attrs = {'units': 'meters', 'long_name': 'Depth at cell center', 'axis': 'Z', 'positive': 'down', 'edges': 'z_i'}
    data.time.attrs = {'units': 'hours since 2010-01-01 00:00:00', 'long_name': 'time', 'axis': 'T', 'calendar_type': 'JULIAN', 'calendar': 'julian'}
    data.xb.attrs = {'long_name': 'Distance along beam from Tasmania towards Macquarie Ridge', 'units': 'km'}
    data.yb.attrs = {'long_name': 'Distance perpendicular to beam referened from beam centre', 'units': 'km'}
    data.attrs["long_name"] = f"filtered velocity data"
    data[["u","v","rho"]].to_netcdf(outputfolder / f"Filtered{offset}.nc",mode="w")
    return 


# if __name__ == "__main__":
#     parser = argparse.ArgumentParser()
#     parser.add_argument('-e', '--expt', type=str, help='Experiment name', default="full-20")
#     parser.add_argument('-t', '--t0', type=int, help='Start time')
#     parser.add_argument('-o', '--offset', type=int, help='Offset from start time',default=-1)
#     parser.add_argument('-w', '--window', type=int, help='Window', default=149)
#     args = parser.parse_args()
#     expt = args.expt


## Defaults if we're just doing all times in one hit
# t0 = args.t0
# filter_window = args.window
# offset = args.offset
# tmpstorage = "/scratch/nm03/ab8992/saphhirerapids"
tmpstorage = os.getenv('PBS_JOBFS')
tmpstorage = "/scratch/nm03/ab8992/troubleshoot_lfilter"
tmpstorage = "/scratch/nm03/ab8992/qsub_lfilter80"
expt = "full-80"
t0 = 12023
filter_window = 233
offset = 6
warnings.filterwarnings("ignore")
sample_window = 333 # 48hrs advection either side of sample window, round 96 up to extra 100
## If we're splitting into multiple jobs for different offsets, adjust times
if offset != -1:
    filter_window = 17 #! Hardcoded for 12 lots of 17 hours + 12 hours
    # sample_window = 150
    start = sample_window // 2 - 117 + offset * filter_window
    if offset == 13:
        filter_window = 12
    stop = start + filter_window
adjusted_t0 = t0 + (stop + start) // 2 - 117 # This gets the middle of the range being filtered
print(f"Filtering from {start} to {stop}")

outputfolder = Path(f"/g/data/nm03/ab8992/postprocessed/{expt}/bandpassed/t0-{t0}")
if not os.path.exists(tmpstorage):
    os.makedirs(tmpstorage)

if not os.path.exists(outputfolder):
    os.makedirs(outputfolder)

# save_data_for_filter(expt,slice(0,96),t0,tmpstorage,sample_window = sample_window) 
# save_data_for_filter(expt,slice(0,2),adjusted_t0,tmpstorage,sample_window = filter_window + 100) #! Alternative: Set sample window to stop - start + 100. Set t0 to the middle of the range. Load less data

## Now, need to pick a new filter range relative to the sampled data set. It needs to filter the middle of the sample_winoow in each case. 
## This will start from 50 and finish at 50 + lfilter_winoow

filter_range = range(
    3600 * 50,
    3600 * (50 + filter_window),
    3600)

print("Data saved. Starting filtering...")
client = startdask(nthreads = 1)
tasks = [delayed(lagrange_filter)(n, tmpstorage,filter_range,filter_window) for n in range(96)] #! PUT 2 BACK TO 96
results = compute(*tasks)

# print("Done. Tidying up...")
# lfilter_tidyup(tmpstorage,outputfolder,offset = offset)


Filtering from 151 to 168
range(543600, 604800, 3600)
Data saved. Starting filtering...
<Client: 'tcp://127.0.0.1:45415' processes=50 threads=50, memory=416.00 GiB>


sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command not found
sh: None: command no

 58% (100800.0 of 172800.0) |#######     | Elapsed Time: 0:01:09 ETA:   0:00:492025-05-22 18:48:42,719 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('lagrange_filter-fe8f7070-c578-4647-8265-7bf75f8760f9')" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2025-05-22 18:48:42,720 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('lagrange_filter-9b19ae4c-8415-4d75-b4fb-7c6be16c8fbe')" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2025-05-22 18:48:42,720 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute('lagrange_filter-d22fc1e0-47f6-46d1-a567-35c4af669

KeyboardInterrupt: 

2025-05-22 18:48:42,915 - distributed.worker - ERROR - Failed to communicate with scheduler during heartbeat.
Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/comm/tcp.py", line 225, in read
    frames_nosplit_nbytes_bin = await stream.read_bytes(fmt_size)
tornado.iostream.StreamClosedError: Stream is closed

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker.py", line 1250, in heartbeat
    response = await retry_operation(
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/utils_comm.py", line 459, in retry_operation
    return await retry(
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/utils_comm.py", line 438, in retry


In [2]:
d = tt.collect_data("beamless-80",rawdata=["u","v","rho"],timerange=(12023 - 50,12023 + 50))

80th run: adjusting outputs to [93 94 95 96 97 98]
loading u...	done.
loading v...	done.
loading rho...	done.


OSError: no files to open

In [8]:
filter_range

range(543600, 604800, 3600)

In [18]:
offset = 6
warnings.filterwarnings("ignore")
sample_window = 333 # 48hrs advection either side of sample window, round 96 up to extra 100
## If we're splitting into multiple jobs for different offsets, adjust times
if offset != -1:
    filter_window = 17 #! Hardcoded for 12 lots of 17 hours + 12 hours
    # sample_window = 150
    start = sample_window // 2 - 117 + offset * filter_window
    if offset == 13:
        filter_window = 12
    stop = start + filter_window
print(start,stop)

151 168


In [ ]:
args:	   (46, '/scratch/nm03/ab8992/qsub_lfilter80', range(666000, 727200, 3600), 17)

In [ ]:

import xarray as xr
xr.open_mfdataset("/g/data/nm03/ab8992/outputs/full-40/output100/u/*")